# 14_all_dramv_output_summaries

This notebook runs dramv parsing functions and loops created in the 13_dramv_functions jupyter notebook.

# Load packages, functions, and data

In [1]:
import pandas as pd
import math
import glob
import os # these two packages are good for searching and navigating file systems
import os.path as op

pd.set_option('display.max_columns', None)

# get_ann_text function
def get_ann_text(hit_text, column_type = 'viral_hit'):
    if type(hit_text) == float: 
        return hit_text
    if column_type == 'viral_hit': 
        no_org = hit_text.split("[")[0] 
        no_acc_id = " ".join(no_org.split(" ")[1:-1])
        return no_acc_id
    if column_type in ['kegg_hit']:
        no_ee = hit_text.split("[")[0].strip()
        return no_ee
    if column_type == 'pfam_hits': 
        no_pf_ids = ";".join([text.split("[")[0].strip() for text in hit_text.split(";")]) 
        return no_pf_ids
    if column_type == 'vogdb_hits': 
        no_code = hit_text.split(";")[0] 
        no_acc = " ".join(no_code.split(" ")[1:]) 
        return no_acc
    
# assign_annot function
def assign_annot(line):
        annot_source = math.nan
        if line['rank'] == 'A' :
            annot_source = 'kegg_hit'
        elif line['rank'] == 'B' :
            annot_source = 'viral_hit'
        elif line['rank'] == 'C' and type(line['kegg_hit']) == str :
            annot_source = 'kegg_hit'
        elif line['rank'] == 'C' and not pd.isna(line['kegg_hit']) and type(line['viral_hit']) == str :
            annot_source = 'viral_hit'
        elif line['rank'] == 'C' and pd.isna(line['kegg_hit']) and type(line['viral_hit']) == str :
            annot_source = 'vogdb_hits'
        elif line['rank'] == 'D' :
            annot_source = 'pfam_hits'
        elif line['rank'] == 'E' and not pd.isna(line['vogdb_hits']) and type(line['vogdb_hits']) == str :
            annot_source = 'vogdb_hits'                                 
        elif line['rank'] == 'E' and pd.isna(line['vogdb_hits']) and type(line['vogdb_hits']) == str :
            annot_source = np.na
        else:
            return math.nan, math.nan
        keep_annot = get_ann_text(line[annot_source], column_type = annot_source)
        return keep_annot, annot_source
        

#dramv_tbls = glob.glob("/Users/melissaherring/Google Drive/My Drive/MH_project/dramv/*/annotations.tsv")

In [3]:
# for loop for running functions

tsv_pattern = "/Users/melissaherring/Google Drive/My Drive/MH_project/dramv/*/annotations.tsv"  # Replace with your file pattern
tsv_file_paths = glob.glob(tsv_pattern)

# Create a list to store the number of columns for each file
num_columns_list = []

# Loop through the list of file paths and count the columns in each TSV file
for file_path in tsv_file_paths:
    df = pd.read_csv(file_path, delimiter='\t')
    num_columns = df.shape[1]
    num_columns_list.append(num_columns)

# Create a DataFrame with file paths and the number of columns
result_df = pd.DataFrame({'File': tsv_file_paths, 'Number of Columns': num_columns_list})
#result_df.to_csv('num_cols.csv')
result_df

,File,Number of Columns
0,/Users/melissaherring/Google Drive/My Drive/MH...,34
1,/Users/melissaherring/Google Drive/My Drive/MH...,21
2,/Users/melissaherring/Google Drive/My Drive/MH...,23
3,/Users/melissaherring/Google Drive/My Drive/MH...,23
4,/Users/melissaherring/Google Drive/My Drive/MH...,23
...,...,...
326,/Users/melissaherring/Google Drive/My Drive/MH...,23
327,/Users/melissaherring/Google Drive/My Drive/MH...,23
328,/Users/melissaherring/Google Drive/My Drive/MH...,27
329,/Users/melissaherring/Google Drive/My Drive/MH...,23


In [ ]:
    tbl['viral_ann_text'] = tbl['viral_hit'].apply(get_ann_text, args = ('viral_hit',))
    tbl['kegg_ann_text'] = tbl['kegg_hit'].apply(get_ann_text, args = ('kegg_hit',)) 
    tbl['pfam_ann_text'] = tbl['pfam_hits'].apply(get_ann_text, args = ('pfam_hits',)) 
    tbl['vogdb_ann_text'] = tbl['vogdb_hits'].apply(get_ann_text, args = ('vogdb_hits',))
    
    tbl[['annotation','annotation_source']] = tbl.apply(assign_annot, axis=1, result_type='expand')